# Question A2 (10 marks)

### In this question, we will determine the optimal batch size for mini-batch gradient descent. Find the optimal batch size for mini-batch gradient descent by training the neural network and evaluating the performances for different batch sizes. Note: Use 5-fold cross-validation on training partition to perform hyperparameter selection. You will have to reconsider the scaling of the dataset during the 5-fold cross validation.

* note: some cells are non-editable and cannot be filled, but leave them untouched. Fill up only cells which are provided.

#### Plot mean cross-validation accuracies on the final epoch for different batch sizes as a scatter plot. Limit search space to batch sizes {128, 256, 512, 1024}. Next, create a table of time taken to train the network on the last epoch against different batch sizes. Finally, select the optimal batch size and state a reason for your selection.

This might take a while to run, so plan your time carefully.

In [1]:
import tqdm
import time
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
from torch import nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from scipy.io import wavfile as wav

from sklearn import preprocessing
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix

from common_utils import set_seed

# setting seed
set_seed()

2. To reduce repeated code, place your

- network (MLP defined in QA1)
- torch datasets (CustomDataset defined in QA1)
- loss function (loss_fn defined in QA1)

in a separate file called **common_utils.py**

Import them into this file. You will not be repenalised for any error in QA1 here as the code in QA1 will not be remarked.

The following code cell will not be marked.

In [2]:
# YOUR CODE HERE
from common_utils import MLP
from common_utils import CustomDataset
from common_utils import loss_fn
from common_utils import split_dataset, preprocess_dataset

df = pd.read_csv('simplified.csv')
df['label'] = df['filename'].str.split('_').str[-2]
label_encoder = preprocessing.LabelEncoder()
df['label'] = label_encoder.fit_transform(df['label'])

X_train = df.drop(['filename', 'label'], axis=1)
y_train = df['label'].to_numpy()
X_train

,tempo,total_beats,average_beats,chroma_stft_mean,chroma_stft_var,chroma_cq_mean,chroma_cq_var,chroma_cens_mean,chroma_cens_var,melspectrogram_mean,...,mfcc15_mean,mfcc15_var,mfcc16_mean,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var
0,184.570312,623,69.222222,0.515281,0.093347,0.443441,0.082742,0.249143,0.021261,0.038422,...,-10.669799,63.340282,1.811605,58.117188,-3.286546,54.268448,-2.719069,59.548176,-4.559987,70.774803
1,151.999081,521,74.428571,0.487201,0.094461,0.542182,0.073359,0.274423,0.008025,0.204988,...,-5.666375,90.256195,1.573594,105.070496,-0.742024,82.417496,-1.961745,119.312355,1.513660,101.014572
2,112.347147,1614,146.727273,0.444244,0.099268,0.442014,0.083224,0.264430,0.013410,0.218063,...,-5.502390,73.079750,0.202623,72.040550,-4.021009,73.844353,-5.916223,103.834824,-2.939086,113.598824
3,107.666016,2060,158.461538,0.454156,0.100834,0.424370,0.084435,0.257672,0.016938,0.214154,...,-8.812989,93.791893,-0.429413,60.002579,-4.013513,82.544540,-5.858006,84.402092,0.686969,90.126389
4,75.999540,66,33.000000,0.478780,0.100000,0.414859,0.089313,0.252143,0.019757,0.128487,...,-6.584204,64.973305,0.744403,68.908516,-6.354805,66.414391,-6.555534,47.852840,-4.809713,73.033966
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12052,86.132812,1605,160.500000,0.549566,0.078633,0.438854,0.078327,0.258500,0.016511,0.008974,...,-12.779257,67.892044,3.546391,49.891026,-3.597707,70.953079,-2.672421,62.369473,-3.812236,47.410625
12053,184.570312,3037,168.722222,0.584372,0.074350,0.478900,0.078894,0.258216,0.016658,0.006628,...,-9.979423,63.430107,3.869859,52.517521,-1.637068,59.811417,-3.041467,55.640205,-5.101839,43.080894
12054,143.554688,1549,129.083333,0.541845,0.088258,0.441677,0.080670,0.261484,0.014959,0.008848,...,-11.238145,79.961853,3.689087,68.597672,-1.665412,70.761398,0.218386,86.220604,-4.678007,85.629585
12055,143.554688,8820,284.516129,0.532886,0.089102,0.469113,0.077342,0.265298,0.012950,0.009798,...,-10.302938,80.636742,2.619849,63.592182,-0.559597,62.905022,0.256377,56.687534,-3.086725,62.594326


3. Define different folds for different batch sizes to get a dictionary of training and validation datasets. Preprocess your datasets accordingly.

In [3]:
def generate_cv_folds_for_batch_sizes(parameters, X_train, y_train):
    cv = KFold(n_splits=5, shuffle=True, random_state=0)
    X_train_scaled_dict, X_val_scaled_dict, y_train_dict, y_val_dict = {}, {}, {}, {}
    for batch_size in parameters:
        X_train_scaled_dict[batch_size] = []
        X_val_scaled_dict[batch_size] = []
        y_train_dict[batch_size] = []
        y_val_dict[batch_size] = []
        for train_idx, test_idx in cv.split(X_train, y_train):
            x_train_fold, y_train_fold = X_train[train_idx], y_train[train_idx]
            
            x_test_fold, y_test_fold = X_train[test_idx], y_train[test_idx]

            x_train_fold_scale, x_test_fold_scale = preprocess_dataset(x_train_fold, x_test_fold)
            
            X_train_scaled_dict[batch_size].append(x_train_fold_scale)
            X_val_scaled_dict[batch_size].append(x_test_fold_scale)
            y_train_dict[batch_size].append(y_train_fold)
            y_val_dict[batch_size].append(y_test_fold)
    """
    returns:
    X_train_scaled_dict(dict) where X_train_scaled_dict[batch_size] is a list of the preprocessed training matrix for the different folds.
    X_val_scaled_dict(dict) where X_val_scaled_dict[batch_size] is a list of the processed validation matrix for the different folds.
    y_train_dict(dict) where y_train_dict[batch_size] is a list of labels for the different folds
    y_val_dict(dict) where y_val_dict[batch_size] is a list of labels for the different folds
    """
    # YOUR CODE HERE
    return X_train_scaled_dict, X_val_scaled_dict, y_train_dict, y_val_dict

batch_sizes = [128, 256, 512, 1024]
X_train_scaled_dict, X_val_scaled_dict, y_train_dict, y_val_dict = generate_cv_folds_for_batch_sizes(batch_sizes, X_train.to_numpy(), y_train)

4. Perform hyperparameter tuning for the different batch sizes with 5-fold cross validation.

In [4]:
# YOUR CODE HERE
no_epochs = 100
from common_utils import EarlyStopper
def find_optimal_hyperparameter(X_train_scaled_dict, X_val_scaled_dict, y_train_dict, y_val_dict, batch_sizes, hyperparameter):
    cross_validation_accuracies, cross_validation_times = [], []
    early_stopper = EarlyStopper(patience=3, min_delta=0)
    for batch_size in batch_sizes:
        print()
        print("Batch Size: " + str(batch_size))
        
        model = MLP(77,128,2)
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
        fold_acc = []
        for fold in range(5):
            train_data = CustomDataset(X_train_scaled_dict[batch_size][fold], y_train_dict[batch_size][fold])
            test_data = CustomDataset(X_val_scaled_dict[batch_size][fold], y_val_dict[batch_size][fold])
            train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
            test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=True)
            epoch_acc = []
            for epoch in range(no_epochs):
                for batch, (x_train,y_train) in enumerate(train_dataloader):
                    pred = model(x_train)
                    loss = loss_fn(pred, y_train)

                    optimizer.zero_grad()
                    loss.backward() 
                    optimizer.step() 
                
                size = len(test_dataloader.dataset)
                correct = 0
                test_loss = 0
                with torch.no_grad():
                    for batch,(x_test, y_test) in enumerate(test_dataloader):
                        pred = model(x_test)
                        test_loss += loss_fn(pred,y_test).item()
                        correct += (pred.argmax(1) == y_test).type(torch.float).sum().item()

                test_loss /= size
                acc = correct/size
                epoch_acc.append(acc)

                if early_stopper.early_stop(test_loss):
                    print("Done!")
                    break

            avg_epoch_acc = np.mean(np.array(epoch_acc), axis = 0)
            print("fold " + str(fold) + ":" + str(avg_epoch_acc))
            fold_acc.append(avg_epoch_acc)
        cross_validation_accuracies.append(np.mean(np.array(fold_acc), axis = 0))

    return cross_validation_accuracies, cross_validation_times
        

batch_sizes = [128, 256, 512, 1024]
cross_validation_accuracies, cross_validation_times = find_optimal_hyperparameter(X_train_scaled_dict, X_val_scaled_dict, y_train_dict, y_val_dict, batch_sizes, 'batch_size')



Batch Size: 128
Done!
fold 0:0.6564400221116639
Done!
fold 1:0.7763266998341625
Done!
fold 2:0.7873289091663211
Done!
fold 3:0.8049564496059726
Done!
fold 4:0.8219618415595189

Batch Size: 256
Done!
fold 0:0.6394254219100576
Done!
fold 1:0.7445066334991708
Done!
fold 2:0.7505184570717546
Done!
fold 3:0.7817295727913729
Done!
fold 4:0.7971795935296557

Batch Size: 512
Done!
fold 0:0.6466324292515916
Done!
fold 1:0.7757048092868988
Done!
fold 2:0.7855661551223558
Done!
fold 3:0.8022604728328495
Done!
fold 4:0.8130236416424719

Batch Size: 1024
Done!
fold 0:0.6252533628155518
Done!
fold 1:0.7338308457711442
Done!
fold 2:0.7388753925460686
Done!
fold 3:0.7745748652011613
Done!
fold 4:0.7870593114890088


5. Plot scatterplot of mean cross validation accuracies for the different batch sizes.

In [5]:
# YOUR CODE HERE
cross_validation_accuracies

[0.7694027844555278, 0.7426719357604024, 0.7646375016272335, 0.731918755564587]

6. Create a table of time taken to train the network on the last epoch against different batch sizes. Select the optimal batch size and state a reason for your selection.

In [ ]:
df = pd.DataFrame({'Batch Size':
                   'Last Epoch Time':
                  })

df

In [ ]:
# YOUR CODE HERE
optimal_batch_size =
reason =